In [1]:
!pip install segmentation_models_pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=875792709a95e69ec32c99b1091a65c826d40ac408f3898e79228b00f1376260
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=09c3e895e6c9bf00f8db6a449c67983ced916a650f28a2e3c571f1ad8041c72f
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientne

In [2]:
import torch
import torch.optim as optim
from torch.jit import load
from collections import OrderedDict
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


/kaggle/input/bkai-igh-neopolyp/sample_submission.csv
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/26f9a7021bab9fc7330398846f67b5df.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/99fdc98376290081835cf877e004e8bf.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/c7abacdbd7e6afd56ea7bc4ba26c86b3.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/bc5d92b9076135d76d0bbd4ce2a1405f.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/f95b204727fdcfa030228a69b99df088.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/6dc55c83a46ce2be9800e6c6f4f21968.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/ad4f13ccf1f4b331a412fc44655fb51f.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/8ee924dbd70f618d52893a23c466c169.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/eb91e2da9e6afaef7fdb2d45b21960c9.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/dec33fd6aab9f0df70e20bccce75a793.jpeg
/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/1ffb963e1713a876

In [3]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)
checkpoint = torch.load('/kaggle/input/unet/pytorch/default/1/model.pth')
model.load_state_dict(checkpoint['model'])
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 229MB/s]
/tmp/ipykernel_30/3645795098.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded

UnetPlusPlus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tru

In [4]:
val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [5]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)  

In [6]:
!mkdir prediction

In [7]:
model.eval()
img_path = '/kaggle/input/bkai-igh-neopolyp/test/test/019410b1fcf0625f608b4ce97629ab55.jpeg'
ori_img = cv2.imread(img_path)
ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
ori_w = ori_img.shape[0]
ori_h = ori_img.shape[1]
img = cv2.resize(ori_img, (256, 256))
transformed = val_transformation(image=img)
input_img = transformed["image"]
input_img = input_img.unsqueeze(0).to(device)
with torch.no_grad():
    output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
mask = cv2.resize(output_mask, (ori_h, ori_w))
mask = np.argmax(mask, axis=2)
mask_rgb = mask_to_rgb(mask, color_dict)
mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
cv2.imwrite("prediction/test_img.jpeg", mask_rgb) 

True

In [8]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    id = 'test_img'
    path = os.path.join(dir, 'test_img.jpeg')
    img = cv2.imread(path)[:,:,::-1]
    h, w = img.shape[0], img.shape[1]
    for channel in range(2):
        ws.append(w)
        hs.append(h)
        ids.append(f'{id}_{channel}')
        string = rle_encode_one_mask(img[:,:,channel])
        strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/prediction'

dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

In [9]:
df.to_csv(r'output.csv', index=False)